# 見出しごとにMarkdownを分割する方法

### 動機

多くのチャットやQ&Aアプリケーションでは、入力ドキュメントを埋め込みやベクトルストレージに送る前にチャンクに分割する必要があります。

Pineconeが提供する以下のメモは参考になります[These notes](https://www.pinecone.io/learn/chunking-strategies/) ：  

- **パラグラフやドキュメント全体を埋め込む場合**  
埋め込みプロセスでは、全体のコンテキストや文やフレーズ間の関係を考慮します。  
その結果、テキストの広義的な意味やテーマをより包括的に表現するベクトル表現が得られます。

- **チャンク化の目的**  
一般的に、同じ文脈を持つテキストを一緒に保つことが重要です。  
この観点から、ドキュメント自体の構造を尊重するのが望ましい場合があります。  
例えば、Markdownファイルは見出しで整理されています。  
この場合、特定の見出しグループごとにチャンクを作成するのが直感的なアプローチです。
 
この課題に対処するために、[MarkdownHeaderTextSplitter](https://python.langchain.com/api_reference/text_splitters/markdown/langchain_text_splitters.markdown.MarkdownHeaderTextSplitter.html) を使用することで、Markdownファイルを指定した見出しごとに分割することができます。

以下のMarkdownを分割したい場合：
```
md = '# Foo\n\n ## Bar\n\nHi this is Jim  \nHi this is Joe\n\n ## Baz\n\n Hi this is Molly' 
```
 
次のような見出しに基づいて分割するよう指定します：
```
[("#", "Header 1"),("##", "Header 2")]
```

これにより、コンテンツが共通の見出しごとにグループ化または分割されます：
```
{'content': 'Hi this is Jim  \nHi this is Joe', 'metadata': {'Header 1': 'Foo', 'Header 2': 'Bar'}}
{'content': 'Hi this is Molly', 'metadata': {'Header 1': 'Foo', 'Header 2': 'Baz'}}
```

以下にいくつかの例を見てみましょう。

### 基本的な使用例:

In [ ]:
%pip install -qU langchain-text-splitters

In [1]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [2]:
markdown_document = "# Foo\n\n    ## Bar\n\nHi this is Jim\n\nHi this is Joe\n\n ### Boo \n\n Hi this is Lance \n\n ## Baz\n\n Hi this is Molly"

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
md_header_splits = markdown_splitter.split_text(markdown_document)
md_header_splits

[Document(page_content='Hi this is Jim  \nHi this is Joe', metadata={'Header 1': 'Foo', 'Header 2': 'Bar'}),
 Document(page_content='Hi this is Lance', metadata={'Header 1': 'Foo', 'Header 2': 'Bar', 'Header 3': 'Boo'}),
 Document(page_content='Hi this is Molly', metadata={'Header 1': 'Foo', 'Header 2': 'Baz'})]

In [3]:
type(md_header_splits[0])

langchain_core.documents.base.Document

デフォルトでは、`MarkdownHeaderTextSplitter` は、分割対象となる見出しを出力チャンクのコンテンツから除去します。  
ただし、`strip_headers=False` を設定することで、この動作を無効化できます。

In [4]:
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on, strip_headers=False)
md_header_splits = markdown_splitter.split_text(markdown_document)
md_header_splits

[Document(page_content='# Foo  \n## Bar  \nHi this is Jim  \nHi this is Joe', metadata={'Header 1': 'Foo', 'Header 2': 'Bar'}),
 Document(page_content='### Boo  \nHi this is Lance', metadata={'Header 1': 'Foo', 'Header 2': 'Bar', 'Header 3': 'Boo'}),
 Document(page_content='## Baz  \nHi this is Molly', metadata={'Header 1': 'Foo', 'Header 2': 'Baz'})]

### Markdownの行を個別のドキュメントとして返す方法

デフォルトでは、`MarkdownHeaderTextSplitter` は、`headers_to_split_on` で指定された見出しに基づいて行を集約します。  
ただし、`return_each_line` を指定することで、この動作を無効化し、各行を個別のドキュメントとして返すことができます。

In [5]:
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on,
    return_each_line=True,
)
md_header_splits = markdown_splitter.split_text(markdown_document)
md_header_splits

[Document(page_content='Hi this is Jim', metadata={'Header 1': 'Foo', 'Header 2': 'Bar'}),
 Document(page_content='Hi this is Joe', metadata={'Header 1': 'Foo', 'Header 2': 'Bar'}),
 Document(page_content='Hi this is Lance', metadata={'Header 1': 'Foo', 'Header 2': 'Bar', 'Header 3': 'Boo'}),
 Document(page_content='Hi this is Molly', metadata={'Header 1': 'Foo', 'Header 2': 'Baz'})]

注意点
- 各行が個別の`Documentオブジェクト`として返されます。
- 見出し情報は、各ドキュメントの`metadata`として保持されます。

### チャンクサイズを制限する方法

Markdownの各グループ内で、`RecursiveCharacterTextSplitter` のようなテキストスプリッターを適用することで、チャンクサイズをさらに制御することができます。

In [6]:
markdown_document = "# Intro \n\n    ## History \n\n Markdown[9] is a lightweight markup language for creating formatted text using a plain-text editor. John Gruber created Markdown in 2004 as a markup language that is appealing to human readers in its source code form.[9] \n\n Markdown is widely used in blogging, instant messaging, online forums, collaborative software, documentation pages, and readme files. \n\n ## Rise and divergence \n\n As Markdown popularity grew rapidly, many Markdown implementations appeared, driven mostly by the need for \n\n additional features such as tables, footnotes, definition lists,[note 1] and Markdown inside HTML blocks. \n\n #### Standardization \n\n From 2012, a group of people, including Jeff Atwood and John MacFarlane, launched what Atwood characterised as a standardisation effort. \n\n ## Implementations \n\n Implementations of Markdown are available for over a dozen programming languages."

headers_to_split_on = [
    ("#", "Header 1"),  # 見出しレベル1
    ("##", "Header 2"),  # 見出しレベル2
]

# Markdownの分割
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on,  # 分割対象の見出しを指定
    strip_headers=False  # 見出しを出力に含める
)
md_header_splits = markdown_splitter.split_text(markdown_document)

# Char-level splits
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size = 250  # 各チャンクの最大文字数
chunk_overlap = 30  # チャンク間で重複する文字数
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# 分割
splits = text_splitter.split_documents(md_header_splits)
splits

[Document(page_content='# Intro  \n## History  \nMarkdown[9] is a lightweight markup language for creating formatted text using a plain-text editor. John Gruber created Markdown in 2004 as a markup language that is appealing to human readers in its source code form.[9]', metadata={'Header 1': 'Intro', 'Header 2': 'History'}),
 Document(page_content='Markdown is widely used in blogging, instant messaging, online forums, collaborative software, documentation pages, and readme files.', metadata={'Header 1': 'Intro', 'Header 2': 'History'}),
 Document(page_content='## Rise and divergence  \nAs Markdown popularity grew rapidly, many Markdown implementations appeared, driven mostly by the need for  \nadditional features such as tables, footnotes, definition lists,[note 1] and Markdown inside HTML blocks.', metadata={'Header 1': 'Intro', 'Header 2': 'Rise and divergence'}),
 Document(page_content='#### Standardization  \nFrom 2012, a group of people, including Jeff Atwood and John MacFarlane,